Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [4]:
!conda install -y gdown

Solving environment: done

# All requested packages already installed.



In [5]:
import gdown

url = 'https://drive.google.com/uc?id=1pFjVgYOrI-bM1xznZ7eRFO2vvBM7q_q1'
output = 'rtoriginal.zip'

gdown.cached_download(url, output, postprocess=gdown.extractall)

File exists: rtoriginal.zip


'rtoriginal.zip'

In [6]:
COLAB_PATH = './RealTimeObjectDetection/'
WORKSPACE_PATH = COLAB_PATH + 'Tensorflow/workspace'
SCRIPTS_PATH = COLAB_PATH + 'Tensorflow/scripts'
APIMODEL_PATH = COLAB_PATH + 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [7]:
labels = [{'name':'hello', 'id':1}, 
          {'name':'yes', 'id':2},
          {'name': 'no', 'id':3},
          {'name': 'thank you', 'id': 4},
          {'name': 'i love you', 'id': 5},]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [8]:
# install object detection api
!apt install -y protobuf-compiler
!python3 -m pip install --upgrade dill==0.3.3 google-api-python-client==1.12.1
!cd ./RealTimeObjectDetection/Tensorflow && rm -r models
!cd RealTimeObjectDetection/Tensorflow && git clone https://github.com/tensorflow/models
!cd RealTimeObjectDetection/Tensorflow/models/research && wget https://github.com/protocolbuffers/protobuf/releases/download/v3.4.0/protoc-3.4.0-linux-x86_64.zip && unzip protoc-3.4.0-linux-x86_64.zip
!cd RealTimeObjectDetection/Tensorflow/models/research && bin/protoc object_detection/protos/*.proto --python_out=.
!cd RealTimeObjectDetection/Tensorflow/models/research && cp object_detection/packages/tf2/setup.py .
!cd RealTimeObjectDetection/Tensorflow/models/research && python3 -m pip install .

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
     |████████████████████████████████| 61 kB 579 kB/s eta 0:00:01
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.8.0
    Uninstalling google-api-python-client-1.8.0:
      Successfully uninstalled google-api-python-client-1.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 1.12.1 which is incompatible.
Cloning into 'models'...
remote: Enumerating objects: 58099, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (167/167), done.
remote: 

In [9]:
!python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: ./RealTimeObjectDetection/Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: ./RealTimeObjectDetection/Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [10]:
# !cd Tensorflow && git clone https://github.com/tensorflow/models

In [11]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [12]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [15]:
!mkdir -p {'RealTimeObjectDetection/Tensorflow/workspace/models/'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

# 5. Update Config For Transfer Learning

In [14]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

### Set TPU or GPU Strategy

In [16]:
try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver().connect() # this eliminates the next two lines of code
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU", tpu.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print("Running on multiple GPUs", [gpu.name for gpu in gpus])
else:
    strategy = tf.distribute.get_strategy()
    print("Running on a single GPU or CPU")
print("Number of accelerators:", strategy.num_replicas_in_sync)

Running on a single GPU or CPU
Number of accelerators: 1


In [17]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [18]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [19]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [20]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [21]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [22]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [23]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python ./RealTimeObjectDetection/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=./RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=./RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


In [24]:
!python3 ./RealTimeObjectDetection/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=./RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=./RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000

2021-06-08 15:26:56.191749: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,
2021-06-08 15:26:59.159542: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so

# 7. Load Train Model From Checkpoint

In [25]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [26]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [27]:
import cv2 
import numpy as np

In [28]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [30]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [32]:
cap.release()

In [33]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

ValueError: in user code:

    <ipython-input-26-f5bf27ce595e>:11 detect_fn  *
        image, shapes = detection_model.preprocess(image)
    /opt/conda/lib/python3.7/site-packages/object_detection/meta_architectures/ssd_meta_arch.py:484 preprocess  *
        normalized_inputs, self._image_resizer_fn)
    /opt/conda/lib/python3.7/site-packages/object_detection/utils/shape_utils.py:492 resize_images_and_return_shapes  *
        outputs = static_or_dynamic_map_fn(
    /opt/conda/lib/python3.7/site-packages/object_detection/utils/shape_utils.py:246 static_or_dynamic_map_fn  *
        outputs = [fn(arg) for arg in tf.unstack(elems)]
    /opt/conda/lib/python3.7/site-packages/object_detection/core/preprocessor.py:3327 resize_image  *
        new_image = tf.image.resize_images(
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/image_ops_impl.py:15

In [34]:
detections = detect_fn(input_tensor)

ValueError: in user code:

    <ipython-input-26-f5bf27ce595e>:11 detect_fn  *
        image, shapes = detection_model.preprocess(image)
    /opt/conda/lib/python3.7/site-packages/object_detection/meta_architectures/ssd_meta_arch.py:484 preprocess  *
        normalized_inputs, self._image_resizer_fn)
    /opt/conda/lib/python3.7/site-packages/object_detection/utils/shape_utils.py:492 resize_images_and_return_shapes  *
        outputs = static_or_dynamic_map_fn(
    /opt/conda/lib/python3.7/site-packages/object_detection/utils/shape_utils.py:246 static_or_dynamic_map_fn  *
        outputs = [fn(arg) for arg in tf.unstack(elems)]
    /opt/conda/lib/python3.7/site-packages/object_detection/core/preprocessor.py:3327 resize_image  *
        new_image = tf.image.resize_images(
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/image_ops_impl.py:15

In [ ]:
from matplotlib import pyplot as plt